In [ ]:
import sys
if '..' not in sys.path:
    sys.path.append('..')

In [ ]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def noise(seq):
    traj = [] 
    toks = tokenizer(seq, max_length=512, truncation=True)['input_ids'][1:] # remove CLS token
    N = len(toks)
    
    while toks:
        toks = toks[:len(toks)//2]
        traj.append(tokenizer.decode(toks))
        
    return traj[::-1], 0, N

In [ ]:
import os
import json
from tqdm import tqdm

lengths = []

with open('../data/imdb/imdb_train_1.jsonl', 'w') as train_f, open('../data/imdb/imdb_dev_1.jsonl', 'w') as dev_f:
    for i, file in tqdm(enumerate(os.listdir('../data/imdb/raw/train/neg'))):
        df = train_f if i < 10000 else dev_f
        with open(f'../data/imdb/raw/train/neg/{file}', 'r') as f:
            seq = f.read()
            *traj, N = noise(seq)
            json.dump(traj, df)
            df.write('\n')
            lengths.append(N)
            
    for i, file in tqdm(enumerate(os.listdir('../data/imdb/raw/train/pos'))):
        df = train_f if i < 10000 else dev_f
        with open(f'../data/imdb/raw/train/pos/{file}', 'r') as f:
            seq = f.read()
            *traj, N = noise(seq)
            json.dump(traj, df)
            df.write('\n')
            lengths.append(N)

In [ ]:
import matplotlib.pyplot as plt

plt.hist(lengths)